In [1]:
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
len(tf.config.list_physical_devices('GPU'))

1

In [3]:
def import_maps(path):
    images = []
    for index, name in enumerate(os.listdir(path)):
        folder = os.path.join(path, name)
        for file_class in os.listdir(folder):
            im_folder = os.path.join(folder, file_class)
            for im in os.listdir(im_folder):
                img = cv2.imread(os.path.join(im_folder, im))
                #print(img.shape)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    img = cv2.resize(img, (224, 224))
                    img = np.dstack((img,img,img))
                if img is not None:
               #     img = (img-np.mean(img))/np.std(img)
                    images.append((np.array(img), index)) 
    return images

In [4]:
image_train_set = import_maps(r'C:\Users\ILBS_14_8_21_Sanjeev\TJK\Train')
image_test_set = import_maps(r'C:\Users\ILBS_14_8_21_Sanjeev\TJK\Test')

In [5]:
len(image_train_set)

15205

In [6]:
len(image_test_set)

2610

In [7]:
train_images_all = [i[0] for i in image_train_set]
train_images_array = np.array(train_images_all)
#train_images_array=np.expand_dims(train_images_array,axis=3)
train_image_label = [i[1] for i in image_train_set]
train_image_label = np.array(train_image_label)

In [8]:
test_images_all = [i[0] for i in image_test_set]
test_images_array = np.array(test_images_all)
test_images_array=np.expand_dims(test_images_array,axis=3)
test_image_label = [i[1] for i in image_test_set]
test_image_label = np.array(test_image_label)

In [9]:
train_images_array.shape

(15205, 224, 224, 3)

In [ ]:
scalar = StandardScaler(copy=False)
train_images = scalar.fit_transform(train_images_array.reshape(15205, 224*224*3))
train_images_array = train_images.reshape(15205, 196, 196, 3)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_images_array,train_image_label,test_size=0.2)

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
train_label_enc = enc.fit_transform(train_y.reshape(-1, 1)).toarray()
val_label_enc = enc.fit_transform(val_y.reshape(-1, 1)).toarray()
test_label_enc = enc.fit_transform(test_image_label.reshape(-1, 1)).toarray()

In [27]:
val_x.shape

(2250, 224, 224, 1)

In [28]:
train_y.shape

(9000,)

In [15]:
from keras.applications.vgg16 import VGG16
model = VGG16()

553476096/553467096 [==============================] - 117s 0us/step


In [16]:
print(model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [34]:
model1 = VGG16(include_top = False, input_shape=(224,224,1))
layer = model1.get_layer('block5_pool').output
x = Flatten(layer)
x = Dropout(0.4)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
out = Dense(3,activation='softmax')(x)
model = Model(model1.input, out)

ValueError: The input must have 3 channels; got `input_shape=(224, 224, 1)`

In [33]:
model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.SGD(0.001),
               metrics=['accuracy'])
history = model.fit(train_x,train_label_enc,validation_data=(val_x,val_label_enc), batch_size=32, epochs=100, verbose=1,shuffle=True)

Epoch 1/100


ValueError: in user code:

    C:\Users\AIIMS-IITD\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\AIIMS-IITD\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\AIIMS-IITD\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\AIIMS-IITD\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\AIIMS-IITD\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\AIIMS-IITD\AppDa